In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import nearest_points
import osm_functions as osf

In [2]:
place = 'Glasgow City, Scotland, UK'

filepath = '/home/ubuntu/strava_data/ubdc_data/Datasets/Strava Metro Assorted/strava_metro_glasgow_2017/'


glasgow_edges = gpd.read_file(filepath + 'glasgow_1k_edges_2017.shp')

# hourly_edge_count = pd.read_csv(filepath + 'glasgow_2017_ride_edge_alignment_hourly.csv')
total_edge_count = pd.read_csv(filepath + 'glasgow_2017_ride_edge_rollup_year_total.csv')


In [3]:
city = ox.gdf_from_place(place)

# generate city grid
city_grid = osf.gridify_polygon(city,0.01)

# Get intersection and streets from graph
graph = ox.graph_from_place(place)
nodes, streets = ox.graph_to_gdfs(graph)
# Some columns contain lists. Convert these to tupes so that they are hashable and can be merged with other dfs
streets['highway'] = streets['highway'].apply(lambda x: tuple(x) if type(x) == list else x)
streets['osmid'] = streets['osmid'].apply(lambda x: tuple(x) if type(x) == list else x)


city

,geometry,place_name,bbox_north,bbox_south,bbox_east,bbox_west
0,"POLYGON ((-4.39320 55.88915, -4.39074 55.88859...","Glasgow City, Scotland, United Kingdom",55.929639,55.781277,-4.071717,-4.393201


In [5]:
glasgow_edges.head()

,id,osm_id,osm_name,osm_meta,osm_source,osm_target,clazz,flags,source,target,km,kmh,cost,reverse_co,x1,y1,x2,y2,geometry
0,45.0,1646.0,Ralston Road,None,4.114840e+05,4.114760e+05,32.0,3.0,76.0,852.0,0.231747,50.0,0.004635,0.004635,-4.336768,55.922714,-4.333213,55.923144,"LINESTRING (-4.33677 55.92271, -4.33625 55.922..."
1,55.0,1735.0,M8,None,1.948010e+05,1.754425e+09,11.0,1.0,86.0,87.0,0.075733,80.0,0.000947,1000000.000000,-4.288174,55.850064,-4.286999,55.849894,"LINESTRING (-4.28817 55.85006, -4.28768 55.849..."
2,56.0,1762.0,None,None,1.637120e+09,2.415333e+09,12.0,1.0,88.0,604068.0,0.028033,80.0,0.000350,1000000.000000,-4.310693,55.849562,-4.310294,55.849447,"LINESTRING (-4.31069 55.84956, -4.31029 55.84945)"
3,57.0,1762.0,None,None,2.415333e+09,1.373877e+09,12.0,1.0,604068.0,89.0,0.379333,80.0,0.004742,1000000.000000,-4.310294,55.849447,-4.304302,55.849141,"LINESTRING (-4.31029 55.84945, -4.30996 55.849..."
4,59.0,1783.0,None,None,3.527720e+05,2.494477e+08,12.0,1.0,92.0,93.0,0.418928,112.0,0.003740,1000000.000000,-4.091932,55.828441,-4.097941,55.830113,"LINESTRING (-4.09419 55.82904, -4.09623 55.829..."


In [6]:
total_edge_count.head()

,edge_id,athcnt_0,rathcnt_0,actcnt_0,ractcnt_0,tathcnt_0,tactcnt_0,acttime_0,racttime_0,cmtcnt_0,...,rathcnt,actcnt,ractcnt,tathcnt,tactcnt,acttime,racttime,cmtcnt,rcmtcnt,tcmtcnt
0,45,0,0,0,0,0,0,NaN,NaN,0,...,30,250,90,65,310,36.0,36.0,180,35,210
1,55,0,0,0,0,0,0,NaN,NaN,0,...,0,20,0,20,20,2.0,NaN,20,0,20
2,56,5,0,10,0,5,10,3.0,NaN,10,...,20,525,20,175,540,2.0,6.0,405,10,410
3,57,0,0,0,0,0,0,NaN,NaN,0,...,10,30,10,25,40,12.0,7.0,25,10,35
4,59,0,0,0,0,0,0,NaN,NaN,0,...,0,25,0,25,25,15.0,NaN,20,0,25


In [8]:
# merge dataframes into a single one for testing
df1 = glasgow_edges[['id','osm_id','geometry']].merge(total_edge_count[['edge_id','tathcnt','tactcnt','tcmtcnt']], left_on='id', right_on='edge_id', how='inner')
df2 = df1.merge(streets[['u','v','osmid','highway','oneway','length']],left_on='osm_id', right_on='osmid', how='inner')

df2 = df2.drop_duplicates(subset='edge_id', keep='first')
df2 = df2.drop(columns=['edge_id','osmid'])

strava_street_counts = df2.copy()
strava_street_counts.head()



,id,osm_id,geometry,tathcnt,tactcnt,tcmtcnt,u,v,highway,oneway,length
0,56.0,1762,"LINESTRING (-4.31069 55.84956, -4.31029 55.84945)",175,540,410,1637120075,2415333077,motorway_link,True,28.033
2,57.0,1762,"LINESTRING (-4.31029 55.84945, -4.30996 55.849...",25,40,35,1637120075,2415333077,motorway_link,True,28.033
4,66.0,1801,"LINESTRING (-4.25000 55.87015, -4.24756 55.869...",35,40,35,1715259260,27456222,motorway,True,32.027
5,67.0,1801,"LINESTRING (-4.24447 55.86965, -4.24265 55.86944)",45,55,45,1715259260,27456222,motorway,True,32.027
6,106.0,1876,"LINESTRING (-4.31275 55.85278, -4.31288 55.852...",600,5105,4165,2285131525,3898424564,primary,True,244.301


In [ ]:
# sum athlete counts in each polygon and get total street length
def street_features(city_grid,strava_street_counts):
    highway_types = ['residential', 'footway', 'service', 'tertiary', 'unclassified',
           'primary', 'cycleway', 'secondary', 'pedestrian', 'path',
           'primary_link', 'steps', 'motorway_link', 'track', 'motorway',
           'secondary_link', 'living_street', 'tertiary_link', 'trunk']
    length_names = [name + '_total_length' for name in highway_types]

    # Initialize out_df that stores strava target and feature data
    out_df = city_grid.copy()
    out_df['grid_tactcnt'] = 0
    out_df = out_df.reindex(out_df.columns.tolist() + length_names, axis=1) 


    for index, row in out_df.iterrows():
        grid_pt = out_df.geometry.iloc[index]
        # Generate boolean list of edge  is in polygon
        indices = strava_street_counts.apply(lambda row: grid_pt.contains(row['geometry']), axis=1)

        # Calculate stats of interrogation polygon
        poly_cyclist_stats = strava_street_counts[indices].groupby('highway').sum()
        # store target data
        out_df['grid_tactcnt'].iloc[index] = poly_cyclist_stats.tactcnt.sum()
        # store feature data
        for c,h_type in enumerate(highway_types):
            try:
                out_df[length_names[c]].iloc[index] = poly_cyclist_stats.length.loc[h_type]
            except:
                out_df[length_names[c]].iloc[index] = np.nan
    return out_df

glasgow_cyclist_count = street_features(city_grid,strava_street_counts)


In [ ]:
glasgow_cyclist_count.head()

In [ ]:
m = folium.Map([city.geometry.centroid.y, city.geometry.centroid.x],
               zoom_start=11,
               tiles="CartoDb positron")

style_city = {'color':'#ebc923 ', 'fillColor': '#ebc923 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city, style_function=lambda x: style_city).add_to(m)

style_region = {'color':'#1FFD09 ', 'fillColor': '#1FFD09 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city_grid.geometry.iloc[64], style_function=lambda x: style_region).add_to(m)

# plot Roads
style_edges = {'color':'#1198f7', 'weight':'1'}
folium.GeoJson(df[indices], style_function=lambda x: style_edges).add_to(m)

style_region = {'color':'#1FFD09 ', 'fillColor': '#1FFD09 ', 'weight':'1', 'fillOpacity' : 0.1}
    
m

In [ ]:
streets[streets.osmid == 1762]